<a href="https://colab.research.google.com/github/Kwaku-code/MSLesSeg/blob/master/Data_preprocessing_(MSLESG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 64.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2

In [ ]:
!pip install keras==2.10.0

In [ ]:
!pip install segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import nibabel as nib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical, Sequence
import matplotlib.pyplot as plt
import random

from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import MeanMetricWrapper
from tensorflow.keras.backend import epsilon

from segmentation_models import Unet
from segmentation_models import get_preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize

Segmentation Models: using `keras` framework.


**Data Loading and Preprocessing**

In [ ]:
# Helper functions
def loadimg(imgpath):
    return nib.load(imgpath).get_fdata()

def cropcentre(imgslice, cropx, cropy):
    y, x = imgslice.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return imgslice[starty:starty + cropy, startx:startx + cropx]

# Data directories
data_dir = './train'
modalities = ['T2', 'FLAIR', 'MASK']

# Load data paths
data_paths = {mod: [] for mod in modalities}
for patient in range(1, 54):
    for timepoint in range(1, 6):
        for mod in modalities:
            path = os.path.join(data_dir, f'P{patient}', f'T{timepoint}', f'P{patient}_T{timepoint}_{mod}.nii.gz')
            if os.path.exists(path):
                data_paths[mod].append(path)
                print(f"Path added: {path}")
            else:
                print(f"Path not found: {path}")

# Extract and save slices for training
def save_slices(data_paths, save_dir, crop_size=(128, 128)):
    os.makedirs(save_dir, exist_ok=True)
    for i, path in enumerate(data_paths['MASK']):
        try:
            imgs = {mod: loadimg(data_paths[mod][i]) for mod in modalities}
            print(f"Processing patient {i}:")
            # Only extract axial slices (axis=2)
            axis = 2
            slices = [np.take(img, indices=range(img.shape[axis]), axis=axis) for img in imgs.values()]
            for j, slice_tuple in enumerate(zip(*slices)):
                slices_dict = {mod: cropcentre(slice, *crop_size) for mod, slice in zip(modalities, slice_tuple)}
                for mod, slice in slices_dict.items():
                    file_path = os.path.join(save_dir, f'{mod}_slice_{i}_{axis}_{j}.npy')
                    np.save(file_path, slice)
                    print(f"Saved: {file_path} with shape {slice.shape}")
        except Exception as e:
            print(f"Error processing patient {i}, timepoint {timepoint}: {e}")

save_dir = './axial_slices'
save_slices(data_paths, save_dir)

# List of all saved slices
all_slices = [os.path.join(save_dir, fname) for fname in os.listdir(save_dir) if fname.endswith('.npy')]
print(f"Number of slices collected: {len(all_slices)}")

# Split training and validation sets
train_slices, val_slices = train_test_split(all_slices, test_size=0.2, random_state=42)
print(f"Training slices: {len(train_slices)}, Validation slices: {len(val_slices)}")

# Save training and validation splits
with open(os.path.join(save_dir, 'train_slices.txt'), 'w') as f:
    for item in train_slices:
        f.write("%s\n" % item)

with open(os.path.join(save_dir, 'val_slices.txt'), 'w') as f:
    for item in val_slices:
        f.write("%s\n" % item)

Streaming output truncated to the last 5000 lines.
Saved: /content/drive/MyDrive/axial_slices/T2_slice_83_2_157.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/FLAIR_slice_83_2_157.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/MASK_slice_83_2_157.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/T2_slice_83_2_158.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/FLAIR_slice_83_2_158.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/MASK_slice_83_2_158.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/T2_slice_83_2_159.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/FLAIR_slice_83_2_159.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/MASK_slice_83_2_159.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/T2_slice_83_2_160.npy with shape (128, 128)
Saved: /content/drive/MyDrive/axial_slices/FLAIR_slice_83_2_160.

In [ ]:
# Number of triplets to use for training and validation
num_train_triplets = 5000
num_val_triplets = 1000

# Data directories
save_dir = './axial_slices'

# Filter slices to get only those with T2, FLAIR, and MASK
t2_slices = [s for s in all_slices if 'T2' in s]
flair_slices = [s for s in all_slices if 'FLAIR' in s]
mask_slices = [s for s in all_slices if 'MASK' in s]

# Ensure number of slices is the same for each modality
min_length = min(len(t2_slices), len(flair_slices), len(mask_slices))
t2_slices = t2_slices[:min_length]
flair_slices = flair_slices[:min_length]
mask_slices = mask_slices[:min_length]

# Combine slices into triplets
triplets = list(zip(t2_slices, flair_slices, mask_slices))

# Split data into training and validation sets
train_triplets, val_triplets = train_test_split(triplets, test_size=0.2, random_state=42)

# Further reduce the number of triplets
train_triplets = train_triplets[:num_train_triplets]
val_triplets = val_triplets[:num_val_triplets]
print(f"Reduced Training triplets: {len(train_triplets)}, Reduced Validation triplets: {len(val_triplets)}")

# Save reduced training and validation splits
with open(os.path.join(save_dir, 'train_triplets_reduced.txt'), 'w') as f:
    for item in train_triplets:
        f.write("%s %s %s\n" % item)

with open(os.path.join(save_dir, 'val_triplets_reduced.txt'), 'w') as f:
    for item in val_triplets:
        f.write("%s %s %s\n" % item)

Reduced Training triplets: 5000, Reduced Validation triplets: 1000


In [ ]:
# Load reduced training and validation triplets
train_triplets_path = os.path.join(save_dir, 'train_triplets_reduced.txt')
val_triplets_path = os.path.join(save_dir, 'val_triplets_reduced.txt')

with open(train_triplets_path, 'r') as f:
    train_triplets = [line.strip().split() for line in f.readlines()]

with open(val_triplets_path, 'r') as f:
    val_triplets = [line.strip().split() for line in f.readlines()]

# Function to load data
def load_data(triplets_list):
    inputs = []
    masks = []
    for t2_path, flair_path, mask_path in triplets_list:
        t2_data = np.load(t2_path)
        flair_data = np.load(flair_path)
        mask_data = np.load(mask_path)
        inputs.append(np.stack([t2_data, flair_data], axis=-1))  # Combine T2 and FLAIR into the last dimension
        masks.append(np.expand_dims(mask_data, axis=-1))  # Add a channel dimension to masks
    inputs = np.array(inputs)
    masks = np.array(masks)
    return inputs, masks

# Load training and validation data
train_inputs, train_masks = load_data(train_triplets)
val_inputs, val_masks = load_data(val_triplets)

# Normalize inputs
train_inputs = train_inputs.astype('float32') / np.max(train_inputs)
val_inputs = val_inputs.astype('float32') / np.max(val_inputs)

# Print shapes of inputs
print(f"train_inputs shape: {train_inputs.shape}")
print(f"train_masks shape: {train_masks.shape}")
print(f"val_inputs shape: {val_inputs.shape}")
print(f"val_masks shape: {val_masks.shape}")

train_inputs shape: (5000, 128, 128, 2)
train_masks shape: (5000, 128, 128, 1)
val_inputs shape: (1000, 128, 128, 2)
val_masks shape: (1000, 128, 128, 1)


In [ ]:
# Data directories
save_dir = './axial_slices'

# Load the reduced training and validation triplets
train_triplets_path = os.path.join(save_dir, 'train_triplets_reduced.txt')
val_triplets_path = os.path.join(save_dir, 'val_triplets_reduced.txt')

with open(train_triplets_path, 'r') as f:
    train_triplets = [line.strip().split() for line in f.readlines()]

with open(val_triplets_path, 'r') as f:
    val_triplets = [line.strip().split() for line in f.readlines()]

# Function to load data
def load_data(triplets_list):
    inputs = []
    masks = []
    for t2_path, flair_path, mask_path in triplets_list:
        t2_data = np.load(t2_path)
        flair_data = np.load(flair_path)
        mask_data = np.load(mask_path)
        inputs.append(np.stack([t2_data, flair_data], axis=-1))  # Combine T2 and FLAIR into the last dimension
        masks.append(np.expand_dims(mask_data, axis=-1))  # Add a channel dimension to masks
    inputs = np.array(inputs)
    masks = np.array(masks)
    return inputs, masks

# Load training and validation data
train_inputs, train_masks = load_data(train_triplets)
val_inputs, val_masks = load_data(val_triplets)

# Normalize inputs
train_inputs = train_inputs.astype('float32') / np.max(train_inputs)
val_inputs = val_inputs.astype('float32') / np.max(val_inputs)

# Print the shapes for debugging
print(f"train_inputs shape: {train_inputs.shape}")
print(f"train_masks shape: {train_masks.shape}")
print(f"val_inputs shape: {val_inputs.shape}")
print(f"val_masks shape: {val_masks.shape}")

train_inputs shape: (5000, 128, 128, 2)
train_masks shape: (5000, 128, 128, 1)
val_inputs shape: (1000, 128, 128, 2)
val_masks shape: (1000, 128, 128, 1)


In [ ]:
# Define paths
save_dir = './axial_train'
train_inputs_path = os.path.join(save_dir, 'train_inputs.npy')
train_masks_path = os.path.join(save_dir, 'train_masks.npy')
val_inputs_path = os.path.join(save_dir, 'val_inputs.npy')
val_masks_path = os.path.join(save_dir, 'val_masks.npy')


# Create directory if non existent
os.makedirs(save_dir, exist_ok=True)

# Save preprocessed data
np.save(train_inputs_path, train_inputs)
np.save(train_masks_path, train_masks)
np.save(val_inputs_path, val_inputs)
np.save(val_masks_path, val_masks)

print(f"Saved train inputs to {train_inputs_path}")
print(f"Saved train masks to {train_masks_path}")
print(f"Saved validation inputs to {val_inputs_path}")
print(f"Saved validation masks to {val_masks_path}")

Saved train inputs to /content/drive/MyDrive/axial_train/train_inputs.npy
Saved train masks to /content/drive/MyDrive/axial_train/train_masks.npy
Saved validation inputs to /content/drive/MyDrive/axial_train/val_inputs.npy
Saved validation masks to /content/drive/MyDrive/axial_train/val_masks.npy


**Load normalised inputs**

In [ ]:
# Define paths
save_dir = './axial_train'
train_inputs_path = os.path.join(save_dir, 'train_inputs.npy')
train_masks_path = os.path.join(save_dir, 'train_masks.npy')
val_inputs_path = os.path.join(save_dir, 'val_inputs.npy')
val_masks_path = os.path.join(save_dir, 'val_masks.npy')

# Load normalized data
if os.path.exists(train_inputs_path) and os.path.exists(train_masks_path) and os.path.exists(val_inputs_path) and os.path.exists(val_masks_path):
    print("Loading normalized data from saved files...")
    train_inputs = np.load(train_inputs_path)
    train_masks = np.load(train_masks_path)
    val_inputs = np.load(val_inputs_path)
    val_masks = np.load(val_masks_path)
else:
    print("Normalized data not found, processing from slices...")
    # Load and process data here if not already saved
    pass

Loading normalized data from saved files...


**TRAINING**